In [1]:
"""
Main libraries
"""
import os
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import tensorflow as tf

import random
import gc

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [2]:
"""
Functions for the onehot encoding
"""

def onehot_encoder(dataset):
    """
    Function that encodes a DNA dataset into a onehot encoding dataset.
    """
    onehot_dataset = [dna_onehot_encoder(dna_string) for dna_string in dataset]
    onehot_dataset_numpy = np.array(onehot_dataset)

    return onehot_dataset_numpy


def dna_onehot_encoder(dna_sequence):
    """
    Function that encodes a single DNA string into a onehot encoding string.
    """
    onehot_dict = {
        'A' : [1, 0, 0, 0],
        'C' : [0, 1, 0, 0],
        'G' : [0, 0, 1, 0],
        'T' : [0, 0, 0, 1]
    }
    encoder = [onehot_dict[nuc] for nuc in dna_sequence]

    return encoder

In [3]:
class LSTM_Net (nn.Module):
    def __init__ (self):
        super(LSTM_Net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)

        self.lstm = nn.LSTM(input_size=128, hidden_size=128, num_layers=6, batch_first=True, dropout=0.5, bidirectional=True)
        # self.lstm = nn.LSTM(input_size=4, hidden_size=128, num_layers=2, dropout=0.5, batch_first=True)

        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)

        x = x[:, -1, :]
        x = x.contiguous().view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

In [4]:
def sequence_to_numeric(sequence):
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    return [mapping[nuc] for nuc in sequence]

# def numeric_to_sequence(numeric_sequence):
#     mapping = ['A', 'C', 'G', 'T']
#     return ''.join([mapping[nuc] for nuc in numeric_sequence])

In [9]:
"""
! MAIN
"""

device = th.device("cuda" if th.cuda.is_available() else "cpu")
print(f"Device: {device}")

data_dir = os.path.abspath(os.path.join(os.getcwd(), 'data'))
rel_path_train = os.path.join(data_dir, 'fullset_train.csv')
rel_path_val = os.path.join(data_dir, 'fullset_validation.csv')
rel_path_test = os.path.join(data_dir, 'fullset_test.csv')

# Training set

# Read the input from the csv file
train_csv = pd.read_csv(rel_path_train, sep=",")
# Drop the NaN values
train_csv = train_csv.dropna()
# Describe the data
# print(train_csv.describe())

# Get the data from the csv file
train_data = train_csv.values
# m = number of input samples
m = train_data.shape[0]

# Dataframe
data = {'sequence' : train_data[:m,1],
        'label' : train_data[:m,2].astype(np.int32) }

df = pd.DataFrame(data)

X_train = df['sequence'].values
y_train = df['label'].values

#Convert the sequences to numeric
numeric_train = [sequence_to_numeric(seq) for seq in X_train]
numeric_train = np.array(numeric_train)

# Reshape the data
n_samples, _ = numeric_train.shape
numeric_train_sequences = numeric_train.reshape((n_samples, -1))

# Apply the SMOTE algorithm to balance the dataset
smote = SMOTE()
X_train, y_train = smote.fit_resample(numeric_train_sequences, y_train)

#Convert the sequences to string
X_train = [''.join([['A', 'C', 'G', 'T'][nuc] for nuc in seq]) for seq in X_train]

# Convert the sequences to onehot encoding
print("Start one hot encoding for training")
X_train = onehot_encoder(X_train)
print("End one hot encoding for training")

X_train = th.from_numpy(X_train).to(device)
Y_train = th.tensor(y_train).to(device)
print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)

# Validation set
# Read the input from the csv file
val_csv = pd.read_csv(rel_path_val, sep=",")
# Drop the NaN values
val_csv = val_csv.dropna()

val_data = val_csv.values
# m = number of input samples
m = val_data.shape[0]

# Dataframe and upsample
data = {'sequence' : val_data[:m,1],
        'label' : val_data[:m,2].astype(np.int32) }

df = pd.DataFrame(data)

X_val = df['sequence'].values
y_val = df['label'].values

#Convert the sequences to numeric
numeric_val = [sequence_to_numeric(seq) for seq in X_val]
numeric_val = np.array(numeric_val)

# Reshape the data
n_samples, _ = numeric_val.shape
numeric_val_sequences = numeric_val.reshape((n_samples, -1))

# Apply the SMOTE algorithm to balance the dataset
smote = SMOTE()
X_val, y_val = smote.fit_resample(numeric_val_sequences, y_val)

#Convert the sequences to string
X_val = [''.join([['A', 'C', 'G', 'T'][nuc] for nuc in seq]) for seq in X_val]

# Convert the sequences to onehot encoding
print("Start one hot encoding for validation")
X_val = onehot_encoder(X_val)
print("End one hot encoding for validation")

X_val = th.from_numpy(X_val).to(device)
Y_val = th.tensor(y_val).to(device)
print("X_val shape: ", X_val.shape)
print("Y_val shape: ", Y_val.shape)

# Test set
# Read the input from the csv file
test_csv = pd.read_csv(rel_path_test, sep=",")
# Drop the NaN values
test_csv = test_csv.dropna()

test_data = test_csv.values
# m = number of input samples
m = test_data.shape[0]

# Dataframe and upsample
data = {'sequence' : test_data[:m,1],
        'label' : test_data[:m,2].astype(np.int32) }

df = pd.DataFrame(data)

X_test = df['sequence'].values
y_test = df['label'].values

#Convert the sequences to numeric
numeric_test = [sequence_to_numeric(seq) for seq in X_test]
numeric_test = np.array(numeric_test)

# Reshape the data
n_samples, _ = numeric_test.shape
numeric_test_sequences = numeric_test.reshape((n_samples, -1))

# Apply the SMOTE algorithm to balance the dataset
smote = SMOTE()
X_test, y_test = smote.fit_resample(numeric_test_sequences, y_test)

#Convert the sequences to string
X_test = [''.join([['A', 'C', 'G', 'T'][nuc] for nuc in seq]) for seq in X_test]

# Convert the sequences to onehot encoding
print("Start one hot encoding for test")
X_test = onehot_encoder(X_test)
print("End one hot encoding for test")

X_test = th.from_numpy(X_test).to(device)
Y_test = th.tensor(y_test).to(device)
print("X_test shape: ", X_test.shape)
print("Y_test shape: ", Y_test.shape)

#free memory
del train_csv, val_csv, test_csv, train_data, val_data, test_data, data, df, numeric_train, numeric_val, numeric_test, numeric_train_sequences, numeric_val_sequences, numeric_test_sequences, 


Device: cpu
Start one hot encoding for training
End one hot encoding for training
X_train shape:  torch.Size([413544, 300, 4])
Y_train shape:  torch.Size([413544])
Start one hot encoding for validation
End one hot encoding for validation
X_val shape:  torch.Size([51740, 300, 4])
Y_val shape:  torch.Size([51740])
Start one hot encoding for test
End one hot encoding for test
X_test shape:  torch.Size([51706, 300, 4])
Y_test shape:  torch.Size([51706])


In [7]:
"""
Class to create a Data Loader with X label and Y label together
"""
class CreateDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_point = self.data[idx]
        label = self.labels[idx]
        return data_point, label

# Create the Dataset
train_dataset = CreateDataset(X_train, Y_train)
val_dataset = CreateDataset(X_val, Y_val)
test_dataset = CreateDataset(X_test, Y_test)

# Batch size
batch_dim = 128

# Create the Data Loader
train_loader = DataLoader(train_dataset, batch_size=batch_dim, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_dim, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_dim, shuffle=True)

# Free memory
del X_train, X_val, Y_val, X_test
gc.collect()
th.cuda.empty_cache()

In [8]:
print("Start training the LSTM")

# Model, loss function and optimizer
model_LSTM = LSTM_Net().to(device)
# criterion = nn.CrossEntropyLoss(weight=class_weights)
criterion = nn.CrossEntropyLoss()
optimizer = th.optim.Adam(model_LSTM.parameters(), lr=0.001)

# Early stopping parameters
patience = 5  # CHECK THE VALUE!!
best_val_loss = float('inf')
counter = 0

# Training the model
num_epochs = 100  # Hope to reach convergence before 100 epoches

for epoch in range(num_epochs):
    model_LSTM.train()
    running_loss = 0.0
    i = 0
    for X_batch, Y_batch in train_loader:

        X_batch = X_batch.float().to(device)
        Y_batch = Y_batch.long().to(device)

        # Forward pass
        outputs = model_LSTM(X_batch.to(device))
        loss = criterion(outputs, Y_batch.to(device))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        i += 1

        """
        scaler = th.cuda.amp.GradScaler()
        with th.cuda.amp.autocast():
          outputs = model_LSTM(X_batch.to(device))
          loss = criterion(outputs, Y_batch.to(device))


        # Backward pass and optimization
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()"""

        # Free memory
        del X_batch, Y_batch
        th.cuda.empty_cache()

    # Validation
    model_LSTM.eval()
    val_loss = 0.0
    j = 0
    with th.no_grad():
        for X_batch, Y_batch in val_loader:
            X_batch = X_batch.float()
            Y_batch = Y_batch.long()

            outputs = model_LSTM(X_batch.to(device))
            loss = criterion(outputs, Y_batch.to(device))

            val_loss += loss.item()
            j += 1

            # Free memory
            del X_batch, Y_batch
            th.cuda.empty_cache()

    # Losses
    running_loss = running_loss/i
    val_loss = val_loss/j

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}')

    # Check for early stopping
    if val_loss <= best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

print("End training the LSTM")

# Free memory
del i, j, running_loss, val_loss
gc.collect()
th.cuda.empty_cache()

Start training the LSTM
Epoch [1/100], Train Loss: 0.1412s
Epoch [2/100], Train Loss: 0.1107s


KeyboardInterrupt: 